In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),]
)

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4, shuffle=False)

In [5]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

n_classes = len(set(trainset.targets))


In [6]:
from torch import nn

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5, 5)),
            nn.MaxPool2d(2),
            nn.PReLU(),
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5)),
        )
        self.dense_layer = nn.Sequential(
            nn.Linear(1600, 120),
            nn.Linear(120, 84),
            nn.Linear(84, n_classes),
        )
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = torch.flatten(out, 1)
        out = self.dense_layer(out)
    
        return out

In [8]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn = CNN()
cnn.to(device)
cnn.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

In [9]:
n_epoch = 2

for epoch in range(n_epoch):
    running_loss = 0.0
    for inputs, labels in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = (
                inputs.to(device),
                labels.to(device),
            )
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print("total loss:", running_loss / len(trainloader))
        

print('Finished Training')

total loss: 1.6046055293393136
total loss: 1.4378046886897087
Finished Training


In [10]:
cnn.eval()
total = 0
correct = 0
for inputs, labels in testloader:
    inputs, labels = (
                inputs.to(device),
                labels.to(device),
            )
    
    outputs = cnn(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print("Test Accuracy: %2.5f" % (correct / total*100))


Test Accuracy: 48.40000
